In [54]:
import tushare as ts
import tensorflow as tf
import os
import sys
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import numpy as np
import datetime
sys.path.append(os.path.join(os.getcwd(), '../common'))
from config import TOKEN
from utils import timer, save, load, datetime2str, get_trade_cal


In [2]:
print(f'tensorflow: {tf.__version__}, tushare: {ts.__version__}')

tensorflow: 1.15.0, tushare: 1.2.85


In [ ]:
pro = ts.pro_api(TOKEN)
print(TOKEN)

In [ ]:
df = pro.index_basic(market='')

In [9]:
pro = ts.pro_api(TOKEN)
df_sh = pro.stock_company(exchange='SSE',
    fields='ts_code,chairman,manager,reg_capital,setup_date,province,main_business,business_scope')
df_sz = pro.stock_company(exchange='SZSE',
    fields='ts_code,chairman,manager,reg_capital,setup_date,province,main_business,business_scope')

In [10]:
df_sh.shape, df_sz.shape

((2259, 8), (2841, 8))

In [11]:
df_sh.head(2)

,ts_code,chairman,manager,reg_capital,setup_date,province,business_scope,main_business
0,603868.SH,李丐腾,李丐腾,43560.0,20060610,上海,剃须刀及配件，家用电器及配件，金属制品的研发、制造、加工；剃须刀及配件、家用电器及配件、金属...,"公司主营业务为个人护理电器产品的研发,生产和销售,以电动剃须刀和电吹风为主要产品."
1,688819.SH,杨建芬,杨建芬,97210.0,20030313,浙江,高性能电池的研发、生产、销售；锂离子电池、燃料电池及其他储能环保电池、新型电极材料的研究开发...,"以电动轻型车动力电池业务为主,集电动特种车动力电池,新能源汽车动力电池,汽车起动启停电池,储..."


In [18]:
df_sz.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2841 entries, 0 to 2840
Data columns (total 8 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ts_code         2841 non-null   object 
 1   chairman        2839 non-null   object 
 2   manager         2807 non-null   object 
 3   reg_capital     2841 non-null   float64
 4   setup_date      2841 non-null   object 
 5   province        2841 non-null   object 
 6   business_scope  2841 non-null   object 
 7   main_business   2832 non-null   object 
dtypes: float64(1), object(7)
memory usage: 177.7+ KB


In [19]:
a=None
"d" in a

TypeError: argument of type 'NoneType' is not iterable

In [20]:
def detect(content):
    if content is None:
        return False
    if "审计" in content:
        return True
    return False


In [23]:
df_sh[df_sh['business_scope'].apply(detect)]

,ts_code,chairman,manager,reg_capital,setup_date,province,business_scope,main_business
1472,601615.SH,张传卫,张传卫,2.272086e+05,20060602,广东,生产经营风力发电主机装备及相关电力电子产品；风电工程技术及风力发电相关技术咨询、技术进出口业...,"主营业务:新能源高端装备制造,新能源电站投资运营及智能管理业务,主要包括:1)大型风力发电机..."
2065,603979.SH,王青海,王心宇,5.954262e+04,20080107,北京,承包与其实力、规模、业绩相适应的国外工程项目；对外派遣实施上述境外工程所需的劳务人员；矿业管...,"金诚信股份是一家集有色金属矿山,黑色金属矿山及化工矿山工程建设与采矿运营管理,矿山设计与技术..."
2138,601018.SH,毛剑宏,宫黎明,1.945439e+06,20080331,浙江,码头开发经营、管理；港口货物的装卸、堆存、仓储、包装、灌装；集装箱拆拼箱、清洗、修理、制造、...,"主要业务:集装箱,铁矿石,原油,煤炭,液体化工,粮食及其他货种装卸及相关业务,同时从事综合物..."


In [24]:
df_sz[df_sz['business_scope'].apply(detect)]

,ts_code,chairman,manager,reg_capital,setup_date,province,business_scope,main_business
1578,002215.SZ,高焕森,高焕森,98954.1427,19990918,广东,农药加工、复配(按国家石油和化学工业局农药生产批准证书办理)，农化产品应用技术研究；农药销售...,"农药制剂产品的研发,生产,销售及植保技术服务.产品包括杀虫剂,杀菌剂,除草剂等三大系列324..."
2494,300675.SZ,叶青,毛洪伟,14666.6700,20070820,广东,一般经营项目：城市及建筑科学研究；城市规划编制；工程咨询、勘察、设计、质量检测与检查、项目管...,"建筑设计,绿色建筑咨询,生态城市规划,公信服务等业务"


In [26]:
df_code2name = pro.concept()

In [35]:
df_code2name['code'].nunique(), df_code2name['name'].nunique()

(879, 879)

In [29]:
df_code2name.head()

,code,name,src
0,TS0,鼠疫,ts
1,TS1,黑龙江自贸区,ts
2,TS2,黑洞概念,ts
3,TS3,黄金水道,ts
4,TS4,黄金概念,ts


In [41]:
df_concept_lst = []
for code in df_code2name['code']:
    df_concept_lst.append(pro.concept_detail(id=code))
df_concept_info = pd.concat(df_concept_lst)

In [43]:
df_concept_info['concept_name'].nunique()

873

In [44]:
df_concept_info.head()

,id,concept_name,ts_code,name
0,TS0,鼠疫,000788.SZ,北大医药
1,TS0,鼠疫,600056.SH,中国医药
2,TS0,鼠疫,600664.SH,哈药股份
3,TS0,鼠疫,600789.SH,鲁抗医药
4,TS0,鼠疫,600812.SH,华北制药


In [51]:
df_concept_info[df_concept_info['concept_name'].str.contains("军工")]

,id,concept_name,ts_code,name
0,TS172,航天军工,000065.SZ,北方国际
1,TS172,航天军工,000099.SZ,中信海直
2,TS172,航天军工,000428.SZ,华天酒店
3,TS172,航天军工,000547.SZ,航天发展
4,TS172,航天军工,000561.SZ,烽火电子
...,...,...,...,...
397,TS709,军工,688418.SH,震有科技
398,TS709,军工,688561.SH,奇安信
399,TS709,军工,688568.SH,中科星图
400,TS709,军工,688586.SH,江航装备


In [55]:
df_concept_info[df_concept_info['concept_name']=="军工"]

,id,concept_name,ts_code,name
0,TS709,军工,000009.SZ,中国宝安
1,TS709,军工,000026.SZ,飞亚达
2,TS709,军工,000066.SZ,中国长城
3,TS709,军工,000070.SZ,特发信息
4,TS709,军工,000099.SZ,中信海直
5,TS709,军工,000400.SZ,许继电气
6,TS709,军工,000425.SZ,徐工机械
7,TS709,军工,000519.SZ,中兵红箭
8,TS709,军工,000534.SZ,万泽股份
9,TS709,军工,000547.SZ,航天发展


In [53]:
df_concept_info[df_concept_info['concept_name'].str.contains("黄金")]

,id,concept_name,ts_code,name
0,TS3,黄金水道,002040.SZ,南京港
1,TS3,黄金水道,002331.SZ,皖通科技
2,TS3,黄金水道,002627.SZ,宜昌交运
3,TS3,黄金水道,600018.SH,上港集团
4,TS3,黄金水道,600119.SH,ST长投
...,...,...,...,...
49,TS4,黄金概念,601212.SH,白银有色
50,TS4,黄金概念,601899.SH,紫金矿业
51,TS4,黄金概念,603900.SH,莱绅通灵
52,TS4,黄金概念,603979.SH,金诚信


In [50]:
df_concept_info[df_concept_info['concept_name'] == "化肥"]

,id,concept_name,ts_code,name
0,TS683,化肥,000422.SZ,ST宜化
1,TS683,化肥,000713.SZ,丰乐种业
2,TS683,化肥,000731.SZ,四川美丰
3,TS683,化肥,000830.SZ,鲁西化工
4,TS683,化肥,000902.SZ,新洋丰
5,TS683,化肥,000912.SZ,泸天化
6,TS683,化肥,002170.SZ,芭田股份
7,TS683,化肥,002312.SZ,三泰控股
8,TS683,化肥,002466.SZ,天齐锂业
9,TS683,化肥,002470.SZ,*ST金正


In [48]:
save(df_concept_info, "/Users/qingbin.zhuang/Personal/StockProject/Data/Important/df_concept_info.pkl")

In [39]:
df

,id,concept_name,ts_code,name
0,TS0,鼠疫,000788.SZ,北大医药
1,TS0,鼠疫,600056.SH,中国医药
2,TS0,鼠疫,600664.SH,哈药股份
3,TS0,鼠疫,600789.SH,鲁抗医药
4,TS0,鼠疫,600812.SH,华北制药


In [3]:
class t:
    a = 3
    def __init__(self, t):
        self.t = t
    def print(self):
        print(f"a: {self.a}, t: {self.t}")
        
class t1(t):
    def __init__(self, t):
        super().__init__(t)
        
h = t1(4)
h.a =44
h.print()

a: 44, t: 4


[tushare](https://tushare.pro/document/2)

In [92]:
pro = ts.pro_api(TOKEN)
print(TOKEN)

64ce1845b91d06f579525db6e53d497b1c513174331f5509320f4bd5


In [98]:
trade_date_lst = get_trade_cal('20220501', '20221215')
trade_date_lst

['20220505',
 '20220506',
 '20220509',
 '20220510',
 '20220511',
 '20220512',
 '20220513',
 '20220516',
 '20220517',
 '20220518',
 '20220519',
 '20220520',
 '20220523',
 '20220524',
 '20220525',
 '20220526',
 '20220527',
 '20220530',
 '20220531',
 '20220601',
 '20220602',
 '20220606',
 '20220607',
 '20220608',
 '20220609',
 '20220610',
 '20220613',
 '20220614',
 '20220615',
 '20220616',
 '20220617',
 '20220620',
 '20220621',
 '20220622',
 '20220623',
 '20220624',
 '20220627',
 '20220628',
 '20220629',
 '20220630',
 '20220701',
 '20220704',
 '20220705',
 '20220706',
 '20220707',
 '20220708',
 '20220711',
 '20220712',
 '20220713',
 '20220714',
 '20220715',
 '20220718',
 '20220719',
 '20220720',
 '20220721',
 '20220722',
 '20220725',
 '20220726',
 '20220727',
 '20220728',
 '20220729',
 '20220801',
 '20220802',
 '20220803',
 '20220804',
 '20220805',
 '20220808',
 '20220809',
 '20220810',
 '20220811',
 '20220812',
 '20220815',
 '20220816',
 '20220817',
 '20220818',
 '20220819',
 '20220822',

In [27]:

df = pro.index_basic(market='')

In [29]:
df

,ts_code,name,market,publisher,category,base_date,base_point,list_date
0,000001.CZC,农期指数,OTH,郑州商品交易所,商品指数,None,1000.0,None
1,000001.SH,上证指数,SSE,中证公司,综合指数,19901219,100.0,19910715
2,000002.CZC,农期谷物,OTH,郑州商品交易所,商品指数,None,1000.0,None
3,000002.SH,上证A指,SSE,中证公司,综合指数,19901219,100.0,19920221
4,000003.CZC,农期软商,OTH,郑州商品交易所,商品指数,None,1000.0,None
...,...,...,...,...,...,...,...,...
7995,h20811.CSI,中国1000材料(全)USD,CSI,中证公司,行业指数,20061229,1000.0,20160510
7996,h20812.CSI,中国1000工业(全)USD,CSI,中证公司,行业指数,20061229,1000.0,20160510
7997,h20813.CSI,中国1000可选(全)USD,CSI,中证公司,行业指数,20061229,1000.0,20160510
7998,h20814.CSI,中国1000消费(全)USD,CSI,中证公司,行业指数,20061229,1000.0,20160510


In [6]:
date='20221209'

In [9]:
data = pro.stock_basic(exchange='', 
                list_status='L', fields='ts_code,symbol,name,area,industry,list_date')

In [11]:
data.groupby('ts_code').size()

ts_code
000001.SZ    1
000002.SZ    1
000004.SZ    1
000005.SZ    1
000006.SZ    1
            ..
873122.BJ    1
873169.BJ    1
873223.BJ    1
873339.BJ    1
873527.BJ    1
Length: 5024, dtype: int64

In [12]:
data.shape

(5024, 6)

In [31]:
data.head(2)

,ts_code,symbol,name,area,industry,list_date
0,000001.SZ,000001,平安银行,深圳,银行,19910403
1,000002.SZ,000002,万科A,深圳,全国地产,19910129


In [13]:
df_price = pro.daily(trade_date=date)
df_money = pro.moneyflow(trade_date=date)
df = pd.merge(df_price, df_money, how = 'outer', 
              on = ['ts_code', 'trade_date'])


In [32]:
dd = pd.merge(data[['ts_code', 'area', 'industry', 'list_date']],
        df, on = 'ts_code', how = 'right')

In [43]:
dd.head(30)

,ts_code,area,industry,list_date,trade_date,open,high,low,close,pre_close,...,buy_lg_vol,buy_lg_amount,sell_lg_vol,sell_lg_amount,buy_elg_vol,buy_elg_amount,sell_elg_vol,sell_elg_amount,net_mf_vol,net_mf_amount
0,000001.SZ,深圳,银行,19910403,20221209,13.40,13.75,13.35,13.70,13.36,...,482569.0,65650.48,421894.0,57377.80,425819.0,58059.87,280653.0,38131.58,231350.0,31802.80
1,000002.SZ,深圳,全国地产,19910129,20221209,19.37,20.63,19.26,20.46,19.38,...,761192.0,152991.66,688741.0,138525.42,671004.0,135294.54,457293.0,91894.89,428199.0,86910.51
2,000004.SZ,深圳,软件服务,19910114,20221209,10.05,10.14,9.95,10.08,10.07,...,1697.0,170.62,2204.0,221.15,0.0,0.00,0.0,0.00,-4050.0,-404.99
3,000005.SZ,深圳,环境保护,19901210,20221209,1.98,1.99,1.91,1.92,2.00,...,38764.0,750.82,64588.0,1250.30,0.0,0.00,0.0,0.00,-72571.0,-1403.86
4,000006.SZ,深圳,区域地产,19920427,20221209,5.26,5.55,5.26,5.45,5.29,...,481836.0,26114.34,445331.0,24127.26,183037.0,9919.56,149809.0,8097.40,241832.0,13200.72
5,000007.SZ,深圳,其他商业,19920413,20221209,8.28,8.38,8.24,8.26,8.31,...,22639.0,1879.54,19472.0,1614.48,1500.0,123.90,2200.0,182.60,4663.0,391.37
6,000008.SZ,北京,运输设备,19920507,20221209,2.51,2.54,2.48,2.50,2.52,...,52326.0,1309.03,73205.0,1828.70,8982.0,224.49,4800.0,120.00,-49669.0,-1234.92
7,000009.SZ,深圳,电气设备,19910625,20221209,13.57,13.67,13.33,13.33,13.70,...,191183.0,25628.31,167246.0,22421.51,59633.0,7989.13,128095.0,17153.25,-161532.0,-21611.51
8,000010.SZ,深圳,建筑工程,19951027,20221209,3.70,3.81,3.66,3.78,3.69,...,43741.0,1645.17,37102.0,1393.09,7103.0,266.07,3888.0,147.36,31967.0,1209.99
9,000011.SZ,深圳,区域地产,19920330,20221209,12.60,12.93,12.60,12.93,11.75,...,25369.0,3258.22,40155.0,5177.90,129675.0,16749.11,27812.0,3586.35,-138122.0,-17888.24


In [55]:
df = get_price_money(date)

=== Begin Read 20221209 data ===
股票OHLS数据:  (5016, 11)
股票基本面数据:  (5016, 17)
股票资金流向:  (4877, 20)
=== End Read 20221209 data ===
股票最终数据:  (5016, 44)


In [58]:
df.head(2)

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv,buy_sm_vol,buy_sm_amount,sell_sm_vol,sell_sm_amount,buy_md_vol,buy_md_amount,sell_md_vol,sell_md_amount,buy_lg_vol,buy_lg_amount,sell_lg_vol,sell_lg_amount,buy_elg_vol,buy_elg_amount,sell_elg_vol,sell_elg_amount,net_mf_vol,net_mf_amount
0,000001.SZ,20221209,13.40,13.75,13.35,13.70,13.36,0.34,2.5449,1615831.92,2197501.649,0.8327,1.8787,1.09,7.3167,6.0616,0.748,1.5696,1.4733,1.3139,1.6642,1.940592e+06,1940554.695,860090.6226,2.658611e+07,2.658560e+07,232453.0,31524.28,438972.0,59703.42,474991.0,64515.54,474313.0,64537.37,482569.0,65650.48,421894.0,57377.80,425819.0,58059.87,280653.0,38131.58,231350.0,31802.80
1,000002.SZ,20221209,19.37,20.63,19.26,20.46,19.38,1.08,5.5728,2177339.59,4380717.630,2.2407,3.3629,1.40,10.5649,10.3980,0.987,0.5255,0.4585,6.1095,4.7351,1.163071e+06,971736.074,647454.9949,2.379643e+07,1.988172e+07,258724.0,51944.34,433293.0,87250.82,486419.0,97841.21,598014.0,120400.63,761192.0,152991.66,688741.0,138525.42,671004.0,135294.54,457293.0,91894.89,428199.0,86910.51


In [52]:
df1 = pro.daily_basic(trade_date=date).drop('close', axis = 1)

In [81]:
df = pro.cyq_perf(trade_date=date)

In [82]:
df.shape

(5000, 11)

In [86]:
pro.cyq_perf(trade_date=date).shape

(5000, 11)

In [84]:
df.head(6)

,ts_code,trade_date,his_low,his_high,cost_5pct,cost_15pct,cost_50pct,cost_85pct,cost_95pct,weight_avg,winner_rate
0,000066.SZ,20221209,0.99,23.99,9.73,10.88,12.03,12.72,13.18,11.83,19.95
1,000026.SZ,20221209,0.87,23.64,8.92,9.61,10.53,11.22,11.68,10.52,53.96
2,000409.SZ,20221209,0.94,22.72,6.44,6.66,6.88,7.32,7.32,6.96,32.15
3,000548.SZ,20221209,0.54,20.94,4.14,4.34,4.74,4.94,4.94,4.74,72.32
4,000429.SZ,20221209,1.25,8.67,5.38,6.01,6.99,7.48,7.76,6.78,91.34
5,000568.SZ,20221209,0.17,324.17,77.93,162.17,188.09,226.97,256.13,188.15,72.43


In [28]:
pro.moneyflow(ts_code='430047.BJ')

,ts_code,trade_date,buy_sm_vol,buy_sm_amount,sell_sm_vol,sell_sm_amount,buy_md_vol,buy_md_amount,sell_md_vol,sell_md_amount,buy_lg_vol,buy_lg_amount,sell_lg_vol,sell_lg_amount,buy_elg_vol,buy_elg_amount,sell_elg_vol,sell_elg_amount,net_mf_vol,net_mf_amount


In [94]:
def get_price_money(date):
    ### 注意, daily 返回 SZ SH BJ的股票 / moneyflow 只返回 SZ SH
    print(f"=== Begin Read {date} data ===")
    # 日线行情 - 股票OHLS数据
    df_price = pro.daily(trade_date=date)
    print("股票OHLS数据: ", df_price.shape)
    
    # 日线行情 - 股票涨跌停数据
    df_limit = pro.stk_limit(trade_date=date)
    print("股票涨跌停数据: ", df_limit.shape)
    
    # 日线行情 - 股票基本面数据
    df_basic = pro.daily_basic(trade_date=date).drop('close', axis = 1)
    print("股票基本面数据: ", df_basic.shape)
    
    # 日线行情 - 股票资金流向
    df_money = pro.moneyflow(trade_date=date)
    print("股票资金流向: ", df_money.shape)
    
    df = pd.merge(df_price, df_limit, how = 'outer', on = ['ts_code', 'trade_date'])
    df = pd.merge(df, df_basic, how = 'outer', on = ['ts_code', 'trade_date'])
    df = pd.merge(df, df_money, how = 'outer', on = ['ts_code', 'trade_date'])

    print(f"=== End Read {date} data ===")
    
    print("股票最终数据: ", df.shape)
    return df

def analysis(data):
    pass

In [89]:
df_trade_money.head(2)

,ts_code,area,industry,list_date,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount,up_limit,down_limit,turnover_rate,turnover_rate_f,volume_ratio,pe,pe_ttm,pb,ps,ps_ttm,dv_ratio,dv_ttm,total_share,float_share,free_share,total_mv,circ_mv,buy_sm_vol,buy_sm_amount,sell_sm_vol,sell_sm_amount,buy_md_vol,buy_md_amount,sell_md_vol,sell_md_amount,buy_lg_vol,buy_lg_amount,sell_lg_vol,sell_lg_amount,buy_elg_vol,buy_elg_amount,sell_elg_vol,sell_elg_amount,net_mf_vol,net_mf_amount
0,000001.SZ,深圳,银行,19910403,20221209,13.40,13.75,13.35,13.70,13.36,0.34,2.5449,1615831.92,2197501.649,14.70,12.02,0.8327,1.8787,1.09,7.3167,6.0616,0.748,1.5696,1.4733,1.3139,1.6642,1.940592e+06,1940554.695,860090.6226,2.658611e+07,2.658560e+07,232453.0,31524.28,438972.0,59703.42,474991.0,64515.54,474313.0,64537.37,482569.0,65650.48,421894.0,57377.80,425819.0,58059.87,280653.0,38131.58,231350.0,31802.80
1,000002.SZ,深圳,全国地产,19910129,20221209,19.37,20.63,19.26,20.46,19.38,1.08,5.5728,2177339.59,4380717.630,21.32,17.44,2.2407,3.3629,1.40,10.5649,10.3980,0.987,0.5255,0.4585,6.1095,4.7351,1.163071e+06,971736.074,647454.9949,2.379643e+07,1.988172e+07,258724.0,51944.34,433293.0,87250.82,486419.0,97841.21,598014.0,120400.63,761192.0,152991.66,688741.0,138525.42,671004.0,135294.54,457293.0,91894.89,428199.0,86910.51


In [99]:
df_static = pro.stock_basic(exchange='', 
                list_status='L', fields='ts_code,symbol,name,area,industry,list_date')
for _d in trade_date_lst:
    df_trade_money = get_price_money(_d)
    df_trade_money = pd.merge(df_static[['ts_code', 'area', 'industry', 'list_date']],
        df_trade_money, on = 'ts_code', how = 'right')
    
    save(df_trade_money, os.path.join(os.getcwd(), f'../DataStore/trade_money/df_trade_money{_d}.pkl'))

=== Begin Read 20220505 data ===
股票OHLS数据:  (4713, 11)
股票涨跌停数据:  (6017, 4)
股票基本面数据:  (4713, 17)
股票资金流向:  (4625, 20)
=== End Read 20220505 data ===
股票最终数据:  (6017, 46)
=== Begin Read 20220506 data ===
股票OHLS数据:  (4748, 11)
股票涨跌停数据:  (6018, 4)
股票基本面数据:  (4748, 17)
股票资金流向:  (4658, 20)
=== End Read 20220506 data ===
股票最终数据:  (6018, 46)
=== Begin Read 20220509 data ===
股票OHLS数据:  (4747, 11)
股票涨跌停数据:  (6018, 4)
股票基本面数据:  (4747, 17)
股票资金流向:  (4659, 20)
=== End Read 20220509 data ===
股票最终数据:  (6018, 46)
=== Begin Read 20220510 data ===
股票OHLS数据:  (4750, 11)
股票涨跌停数据:  (6018, 4)
股票基本面数据:  (4750, 17)
股票资金流向:  (4662, 20)
=== End Read 20220510 data ===
股票最终数据:  (6018, 46)
=== Begin Read 20220511 data ===
股票OHLS数据:  (4750, 11)
股票涨跌停数据:  (6019, 4)
股票基本面数据:  (4750, 17)
股票资金流向:  (4662, 20)
=== End Read 20220511 data ===
股票最终数据:  (6019, 46)
=== Begin Read 20220512 data ===
股票OHLS数据:  (4751, 11)
股票涨跌停数据:  (6022, 4)
股票基本面数据:  (4751, 17)
股票资金流向:  (4663, 20)
=== End Read 20220512 data ===
股票最终数据:  (6022, 46

股票OHLS数据:  (4822, 11)
股票涨跌停数据:  (6059, 4)
股票基本面数据:  (4822, 17)
股票资金流向:  (4719, 20)
=== End Read 20220714 data ===
股票最终数据:  (6059, 46)
=== Begin Read 20220715 data ===
股票OHLS数据:  (4826, 11)
股票涨跌停数据:  (6064, 4)
股票基本面数据:  (4826, 17)
股票资金流向:  (4723, 20)
=== End Read 20220715 data ===
股票最终数据:  (6064, 46)
=== Begin Read 20220718 data ===
股票OHLS数据:  (4829, 11)
股票涨跌停数据:  (6066, 4)
股票基本面数据:  (4829, 17)
股票资金流向:  (4725, 20)
=== End Read 20220718 data ===
股票最终数据:  (6066, 46)
=== Begin Read 20220719 data ===
股票OHLS数据:  (4828, 11)
股票涨跌停数据:  (6070, 4)
股票基本面数据:  (4828, 17)
股票资金流向:  (4724, 20)
=== End Read 20220719 data ===
股票最终数据:  (6070, 46)
=== Begin Read 20220720 data ===
股票OHLS数据:  (4828, 11)
股票涨跌停数据:  (6071, 4)
股票基本面数据:  (4828, 17)
股票资金流向:  (4724, 20)
=== End Read 20220720 data ===
股票最终数据:  (6071, 46)
=== Begin Read 20220721 data ===
股票OHLS数据:  (4828, 11)
股票涨跌停数据:  (6074, 4)
股票基本面数据:  (4828, 17)
股票资金流向:  (4724, 20)
=== End Read 20220721 data ===
股票最终数据:  (6074, 46)
=== Begin Read 20220722 data ==

股票涨跌停数据:  (6193, 4)
股票基本面数据:  (4919, 17)
股票资金流向:  (4808, 20)
=== End Read 20220922 data ===
股票最终数据:  (6193, 46)
=== Begin Read 20220923 data ===
股票OHLS数据:  (4922, 11)
股票涨跌停数据:  (6195, 4)
股票基本面数据:  (4922, 17)
股票资金流向:  (4810, 20)
=== End Read 20220923 data ===
股票最终数据:  (6195, 46)
=== Begin Read 20220926 data ===
股票OHLS数据:  (4926, 11)
股票涨跌停数据:  (6200, 4)
股票基本面数据:  (4926, 17)
股票资金流向:  (4814, 20)
=== End Read 20220926 data ===
股票最终数据:  (6200, 46)
=== Begin Read 20220927 data ===
股票OHLS数据:  (4930, 11)
股票涨跌停数据:  (6204, 4)
股票基本面数据:  (4930, 17)
股票资金流向:  (4816, 20)
=== End Read 20220927 data ===
股票最终数据:  (6204, 46)
=== Begin Read 20220928 data ===
股票OHLS数据:  (4934, 11)
股票涨跌停数据:  (6207, 4)
股票基本面数据:  (4934, 17)
股票资金流向:  (4819, 20)
=== End Read 20220928 data ===
股票最终数据:  (6207, 46)
=== Begin Read 20220929 data ===
股票OHLS数据:  (4935, 11)
股票涨跌停数据:  (6210, 4)
股票基本面数据:  (4935, 17)
股票资金流向:  (4820, 20)
=== End Read 20220929 data ===
股票最终数据:  (6210, 46)
=== Begin Read 20220930 data ===
股票OHLS数据:  (4936, 11

股票基本面数据:  (5010, 17)
股票资金流向:  (4874, 20)
=== End Read 20221207 data ===
股票最终数据:  (6306, 46)
=== Begin Read 20221208 data ===
股票OHLS数据:  (5014, 11)
股票涨跌停数据:  (6311, 4)
股票基本面数据:  (5014, 17)
股票资金流向:  (4877, 20)
=== End Read 20221208 data ===
股票最终数据:  (6311, 46)
=== Begin Read 20221209 data ===
股票OHLS数据:  (5016, 11)
股票涨跌停数据:  (6316, 4)
股票基本面数据:  (5016, 17)
股票资金流向:  (4877, 20)
=== End Read 20221209 data ===
股票最终数据:  (6316, 46)
=== Begin Read 20221212 data ===
股票OHLS数据:  (5022, 11)
股票涨跌停数据:  (6320, 4)
股票基本面数据:  (5022, 17)
股票资金流向:  (4882, 20)
=== End Read 20221212 data ===
股票最终数据:  (6320, 46)
=== Begin Read 20221213 data ===
股票OHLS数据:  (5023, 11)
股票涨跌停数据:  (6322, 4)
股票基本面数据:  (5023, 17)
股票资金流向:  (4882, 20)
=== End Read 20221213 data ===
股票最终数据:  (6322, 46)
=== Begin Read 20221214 data ===
股票OHLS数据:  (5024, 11)
股票涨跌停数据:  (6325, 4)
股票基本面数据:  (5024, 17)
股票资金流向:  (4881, 20)
=== End Read 20221214 data ===
股票最终数据:  (6325, 46)
=== Begin Read 20221215 data ===
股票OHLS数据:  (5028, 11)
股票涨跌停数据:  (6329, 4

In [29]:
a.year

2022

In [8]:
pd.date_range(start='20221101', end = '20221201')

DatetimeIndex(['2022-11-01', '2022-11-02', '2022-11-03', '2022-11-04',
               '2022-11-05', '2022-11-06', '2022-11-07', '2022-11-08',
               '2022-11-09', '2022-11-10', '2022-11-11', '2022-11-12',
               '2022-11-13', '2022-11-14', '2022-11-15', '2022-11-16',
               '2022-11-17', '2022-11-18', '2022-11-19', '2022-11-20',
               '2022-11-21', '2022-11-22', '2022-11-23', '2022-11-24',
               '2022-11-25', '2022-11-26', '2022-11-27', '2022-11-28',
               '2022-11-29', '2022-11-30', '2022-12-01'],
              dtype='datetime64[ns]', freq='D')

In [27]:
pd.date_range?

In [5]:
with timer("Read Data"):
    df_trade_money = get_price_money('20221201')

股票交易数据:  (5008, 11)
股票资金流向:  (4876, 20)
[Read Data] done in 3.53 s


In [6]:
df_trade_money.shape

(5008, 29)

In [8]:
save(df_trade_money, os.path.join(os.getcwd(), '../DataStore/20221201/df_trade_money.pkl'))

/Users/qingbin.zhuang/Personal/StockProject/Debug/../DataStore/20221201 not exists, create new folder ...


In [9]:
df_trade_money.columns

Index(['ts_code', 'trade_date', 'open', 'high', 'low', 'close', 'pre_close',
       'change', 'pct_chg', 'vol', 'amount', 'buy_sm_vol', 'buy_sm_amount',
       'sell_sm_vol', 'sell_sm_amount', 'buy_md_vol', 'buy_md_amount',
       'sell_md_vol', 'sell_md_amount', 'buy_lg_vol', 'buy_lg_amount',
       'sell_lg_vol', 'sell_lg_amount', 'buy_elg_vol', 'buy_elg_amount',
       'sell_elg_vol', 'sell_elg_amount', 'net_mf_vol', 'net_mf_amount'],
      dtype='object')

In [ ]:
df_trade_money

In [14]:
df_trade_money.sort_values('amount', ascending=False)[['ts_code', 'pct_chg', 'amount']].head(10)

,ts_code,pct_chg,amount
3135,600519.SH,1.9887,8480772.923
2220,300750.SZ,2.6623,8341109.691
189,000625.SZ,2.2305,8330247.561
339,000858.SZ,4.9280,7325021.108
1554,300059.SZ,2.8649,7084272.140
1082,002594.SZ,2.7512,6927128.718
1091,002603.SZ,-5.4343,6347245.186
2766,600050.SH,7.0295,5453223.212
3701,601888.SH,3.3016,4952023.694
3532,601012.SH,2.6868,4543057.814


In [23]:
df_trade_money[(not df_trade_money.ts_code.str.startswith('3')) && (not df_trade_money.ts_code.str.startswith('6'))]

SyntaxError: invalid syntax (2334402625.py, line 1)

In [60]:
# 同花顺概念和行业指数
df_industry = pro.ths_index()
print(df_industry.columns, df_industry.shape)
df_industry.head(2)

Index(['ts_code', 'name', 'count', 'exchange', 'list_date', 'type'], dtype='object') (1653, 6)


,ts_code,name,count,exchange,list_date,type
0,864005.TI,区块链,25.0,A,20200102,N
1,864006.TI,固态电池,3.0,A,20200102,N


In [70]:
df_industry.sort_values('count').head(2)

,ts_code,name,count,exchange,list_date,type
1300,877101.TI,烟草,1.0,HK,20150101,I
1025,861153.TI,海港与服务,1.0,US,None,I


In [63]:
df_industry['ts_code'].nunique()

1653

In [71]:
# 
df = pro.ths_daily(ts_code='877101.TI', 
                   start_date='20220907', end_date='20220907', fields='ts_code,trade_date,open,close,high,low,pct_change')

In [73]:
df

,ts_code,trade_date,close,open,high,low,pct_change


In [74]:
# 
df = pro.ths_daily(start_date='20220907', end_date='20220907', fields='ts_code,trade_date,open,close,high,low,pct_change')

In [75]:
df.shape

(1467, 7)

In [66]:
df['ts_code'].nunique()

1054

In [33]:
start_date, end_date = '20220101', '20220907'
ts_code = '600096.SH'


In [38]:
df_price_all = pro.daily(trade_date=end_date)

In [40]:
df_price_all.head(2)


,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
0,000001.SZ,20220907,12.42,12.42,12.28,12.33,12.50,-0.17,-1.3600,979812.81,1208329.759
1,000002.SZ,20220907,17.66,17.82,17.40,17.52,17.85,-0.33,-1.8487,883386.10,1550325.356


In [43]:
df_price_all.sort_values('amount', ascending=False).head(10)

,ts_code,trade_date,open,high,low,close,pre_close,change,pct_chg,vol,amount
2209,300750.SZ,20220907,451.49,476.49,450.00,473.92,455.20,18.72,4.1125,216777.12,1.009527e+07
1072,002594.SZ,20220907,272.00,285.05,270.60,282.70,272.94,9.76,3.5759,190873.17,5.345728e+06
3021,600438.SH,20220907,56.00,58.02,55.98,56.69,56.87,-0.18,-0.3165,919203.91,5.238977e+06
3079,600519.SH,20220907,1835.03,1837.66,1811.01,1818.01,1845.00,-26.99,-1.4629,28306.16,5.151537e+06
948,002466.SZ,20220907,109.00,110.88,108.26,109.51,110.77,-1.26,-1.1375,390896.42,4.274902e+06
3476,601012.SH,20220907,53.00,54.28,52.71,53.46,53.16,0.30,0.5643,713985.67,3.842027e+06
1749,300274.SZ,20220907,121.98,126.60,121.10,122.16,122.59,-0.43,-0.3508,274861.51,3.405456e+06
1502,300014.SZ,20220907,94.00,99.60,93.36,98.49,94.80,3.69,3.8924,318640.79,3.107963e+06
856,002371.SZ,20220907,295.98,316.60,295.01,315.00,290.00,25.00,8.6207,101340.40,3.105520e+06
568,002077.SZ,20220907,16.00,17.66,15.87,17.66,16.05,1.61,10.0312,1627182.29,2.819322e+06


In [27]:
# 日线行情 - 获取单个股票交易数据
# df = pro.daily(trade_date='20180810')
df_price = pro.daily(ts_code=ts_code, start_date=start_date, end_date=end_date)
print(df_price.columns, df_price.shape)

Index(['ts_code', 'trade_date', 'open', 'high', 'low', 'close', 'pre_close',
       'change', 'pct_chg', 'vol', 'amount'],
      dtype='object') (166, 11)


In [28]:
# 个股资金流向 - 获取单个股票交易数据
df_trade = pro.moneyflow(ts_code=ts_code, start_date=start_date, end_date=end_date)
print(df_trade.columns, df_trade.shape)

Index(['ts_code', 'trade_date', 'buy_sm_vol', 'buy_sm_amount', 'sell_sm_vol',
       'sell_sm_amount', 'buy_md_vol', 'buy_md_amount', 'sell_md_vol',
       'sell_md_amount', 'buy_lg_vol', 'buy_lg_amount', 'sell_lg_vol',
       'sell_lg_amount', 'buy_elg_vol', 'buy_elg_amount', 'sell_elg_vol',
       'sell_elg_amount', 'net_mf_vol', 'net_mf_amount'],
      dtype='object') (166, 20)


In [29]:
df = pd.merge(df_price, df_trade, how = 'outer', on = ['ts_code', 'trade_date'])

In [30]:
df.shape

(166, 29)

In [32]:
df[['trade_date', 'vol', 'amount', 'net_mf_vol', 'net_mf_amount']].head()

,trade_date,vol,amount,net_mf_vol,net_mf_amount
0,20220907,518811.74,1351711.454,-28218,-7217.54
1,20220906,906807.09,2375990.965,170965,44808.18
2,20220905,594687.23,1454065.589,62140,15432.26
3,20220902,650055.50,1624532.303,-143712,-35905.69
4,20220901,401049.07,1044564.477,-117133,-30434.12


In [17]:
df.tail()

,ts_code,trade_date,buy_sm_vol,buy_sm_amount,sell_sm_vol,sell_sm_amount,buy_md_vol,buy_md_amount,sell_md_vol,sell_md_amount,buy_lg_vol,buy_lg_amount,sell_lg_vol,sell_lg_amount,buy_elg_vol,buy_elg_amount,sell_elg_vol,sell_elg_amount,net_mf_vol,net_mf_amount
157,600096.SH,20220110,171057,31652.57,167801,31158.94,183072,33868.65,182960,33856.71,116421,21554.52,107504,19832.79,16106,2995.38,28390,5222.68,16216,3207.25
158,600096.SH,20220107,184676,35017.02,136438,25884.78,129571,24573.33,125081,23713.90,66781,12678.35,90180,17095.39,14098,2687.83,43427,8262.46,-77201,-14557.85
159,600096.SH,20220106,166838,32061.25,182223,35050.13,156900,30131.20,168811,32437.05,95499,18355.48,79616,15302.15,21228,4119.97,9814,1878.57,86931,16789.16
160,600096.SH,20220105,166863,31701.32,132610,25210.83,119623,22737.46,126584,24075.15,70808,13456.44,85560,16265.11,28658,5470.52,41198,7814.66,-93691,-17734.56
161,600096.SH,20220104,199176,38465.86,170836,33090.94,190236,36794.90,183913,35552.50,121345,23517.01,133501,25816.51,19431,3771.18,41938,8089.00,-11252,-2072.21


In [ ]:
# 每日涨跌停价格
#获取单日全部股票数据涨跌停价格
# df = pro.stk_limit(trade_date='20190625')

#获取单个股票数据
# df = pro.stk_limit(ts_code='002149.SZ', start_date='20190115', end_date='20190615')

In [34]:
df_most = pro.stk_limit(trade_date=end_date)

In [35]:
df_most.columns, df_most.shape

(Index(['trade_date', 'ts_code', 'up_limit', 'down_limit'], dtype='object'),
 (5800, 4))

In [36]:
df_most

,trade_date,ts_code,up_limit,down_limit
0,20220907,000001.SZ,13.75,11.25
1,20220907,000002.SZ,19.64,16.07
2,20220907,000004.SZ,9.87,8.93
3,20220907,000005.SZ,1.96,1.78
4,20220907,000006.SZ,4.92,4.02
...,...,...,...,...
5795,20220907,688295.SH,51.96,34.64
5796,20220907,688296.SH,27.06,18.04
5797,20220907,688297.SH,58.50,39.00
5798,20220907,688298.SH,107.03,71.35


In [1]:
class T:
    def __init__(self):
        self.a = 1

In [2]:
a: T

In [3]:
a = 1

In [9]:
def test(a: int, b: str):
    return a+b

In [10]:
test(1.0,2)

3.0